# Multisig Wallet tutorial and playground with web3.js
Tested on Energy Web Foundation's Volta test network and ganache.


On a live test network please make 3 test accounts with some test tokens for experimentation

In [ ]:
const Web3 = require('web3');
const fs = require("fs");

// Deployed factory address on Volta. Feel free to change it to yours.
const factoryWithDLAddress = "0xA76bAb23973EFc2A456572aE6dc10991260Df3e3"

const pathRoot = "node_modules/multisig-wallet-gnosis/build/contracts/"
const factoryWithDLAbi = JSON.parse(fs.readFileSync(pathRoot + "MultiSigWalletWithDailyLimitFactory.json"))["abi"]
const walletWithDLJSON = JSON.parse(fs.readFileSync(pathRoot + "MultiSigWalletWithDailyLimit.json"))

const ADDRESS_EMPTY = '0x0000000000000000000000000000000000000000'

In [ ]:
const provider = new Web3.providers.HttpProvider("http://localhost:8545")
const web3 = new Web3(provider)

let address1, address2, address3

// you should have 3 unlocked accounts to play around
web3.eth.personal.getAccounts().then((accounts) => {
    console.log(accounts)
    address1 = accounts[0]
    address2 = accounts[1]
    address3 = accounts[2]
})
//web3.eth.personal.unlockAccount(w3.eth.defaultAccount, "passwordhere", duration)

### 1. Let's create a multisig wallet
You need:
 - the owner account addresses
 - how many confirmations are needed to perform transactions
 - daily limit -> the amount that can be withrdrawn per day without the confirmation of others

These setting can be later changed

2 ways to create the wallet:
 1. A: using the deployed wallet factory (simpler)
 2. B: compiling and deploying the Wallet contract yourself

Choose whichever fits you the best

### 1.A Using a factory

#### Let's create a factory and wallet instance

In [ ]:
let factoryWithDailyLimit
let requiredConfirmations
let dailyLimit
let txHash

In [ ]:
// factory is already deployed so we know the address
factoryWithDailyLimit = new web3.eth.Contract(factoryWithDLAbi, factoryWithDLAddress)
console.log("---")

In [ ]:
requiredConfirmations = 2
dailyLimit = 0

factoryWithDailyLimit.methods.create([address1, address2, address3], requiredConfirmations, dailyLimit)
    .send({"from": address1, "gas": 5000000})
    .then((x) => {
        txHash=x
        console.log(txHash)
    })

The factory emits a ```ContractInstantiation(address sender, address instantiation)``` event in case of a newly created wallet. We can get the wallet's address by accessing it.
You can either read the event logs and parse the data from the transaction receipt, or set a filter for the event and scan for it. The first method is shown. Then the wallet contract is then instantiated using the address and its ABI.

In [ ]:
let myWalletAddress
let myWallet

In [ ]:
myWalletAddress = txHash.events.ContractInstantiation.returnValues.instantiation
myWallet = new web3.eth.Contract(walletWithDLAbi, myWalletAddress)

console.log("Your deployed wallet address is: " + myWalletAddress)
myWallet.methods.getOwners().call().then((x) => { console.log("Owners are: " + x)} )
myWallet.methods.dailyLimit().call().then((x) => { console.log("Daily withdraw limit w/o confirmations: " + x + " wei") })
myWallet.methods.calcMaxWithdraw().call().then((x) => { console.log("Allowed withdraw for today w/o confirmations: " + x + " wei") })
myWallet.methods.required().call().then((x) => { console.log("Required confirmations: " + x) })

### 1.B deploying the wallet manually

In [ ]:
let MyWallet
let myWallet
let requiredConfirmations
let dailyLimit

In [ ]:
MyWallet = new web3.eth.Contract(walletWithDLJSON.abi)

requiredConfirmations = 2
dailyLimit = 0

MyWallet.deploy({
    data: walletWithDLJSON.unlinked_binary, //walletWithDLJSON.bytecode with newer compilers
    arguments: [[address1, address2, address3], requiredConfirmations, dailyLimit]
}).send({"from": address1, "gas": 5000000})
    .then(x => {
        myWallet = x
        console.log("Contract creation success")
    })

In [ ]:
console.log("Your deployed wallet address is: " + myWallet.options.address)
myWallet.methods.getOwners().call().then((x) => { console.log("Owners are: " + x)} )
myWallet.methods.dailyLimit().call().then((x) => { console.log("Daily withdraw limit w/o confirmations: " + x + " wei") })
myWallet.methods.calcMaxWithdraw().call().then((x) => { console.log("Allowed withdraw for today w/o confirmations: " + x + " wei") })
myWallet.methods.required().call().then((x) => { console.log("Required confirmations: " + x) })

### 2. Let's send some play tokens to the wallet
E.g. 2 ethers for playing around

In [ ]:
web3.eth.sendTransaction({
    "from": address1,
    "to": myWalletAddress,
    "value": web3.utils.toWei("0.5", "ether")
})

### 3. Let's try to withdraw some money back to account 1
 1. We submit a transaction invoking the ```submitTransaction(address destination, uint value, bytes data)```. ```value``` is where the transferrable 'money' goes in wei. More about the ```data``` field below in secton 5.
 2. We need to get the transaction ID: the wallet emits a ```Submission(uint indexed transactionId)``` event in case of  successful submission. We can read it out from the receipt.

In [ ]:
let txHash2
let transactionId

In [ ]:
myWallet.methods.submitTransaction(address1, web3.utils.toWei("0.2", "ether"), "0x0")
    .send({"from": address1})
    .then(x => {
        txHash2=x
        transactionId = x.events.Submission.returnValues.transactionId
        console.log("Submission success! Transaction ID:" + transactionId)
    })

#### 3.1 Let's check out the state of our submission

In [ ]:
myWallet.methods.transactionCount().call().then((x) => console.log(`Transaction count: ${x}`))
myWallet.methods.transactions(transactionId).call().then((x) => console.log(`Our transaction status: ${JSON.stringify(x)}`))
myWallet.methods.confirmations(transactionId, address1).call().then((x) => console.log(`Confirmed by ${address1}: ${x}`))
myWallet.methods.confirmations(transactionId, address2).call().then((x) => console.log(`Confirmed by ${address2}: ${x}`))
myWallet.methods.confirmations(transactionId, address3).call().then((x) => console.log(`Confirmed by ${address3}: ${x}`))

#### 3.2 You can check that as long as the submission is not confirmed by at least another owner, you cannot send it

In [ ]:
myWallet.methods.executeTransaction(transactionId).send({"from": address1, "gas": 800000})

In [ ]:
myWallet.methods.transactions(transactionId).call().then(x => console.log(`Our transaction status: ${JSON.stringify(x)}`))

### 4. Confirm the transaction with other owners
 - ```confirmTransaction(uint transactionId)```
 - needs the transaction ID, and the sender needs to be the owner who confirms
 - a confirmed transaction can be executed by ``` executeTransaction(transactionId)```
 - ``` executeTransaction(transactionId)``` is automatically triggered if the number of confirmations reach the required with this last ```confirmTransaction``` and all conditions are met


In [ ]:
myWallet.methods.confirmTransaction(transactionId).send({"from": address2, "gas": 800000})

In [ ]:
myWallet.methods.transactions(transactionId).call().then(x => console.log(`Our transaction status: ${JSON.stringify(x)}`))

You can see that the transaction status is executed and the money appears on you destination account.

**Note**: If the last confirmer didn't send enough gas to execute the transaction, the execution has to be fired manually using the executeTransaction(id) function

### 5. Change daily limit 
You cannot calll the wallet's changeDailyLimit function directly, it needs to be a confirmed transaction. You can change the daily limit, owners and required confirmations as well, with the consent of the other owners.

#### How to invoke Smart Contract methods using your Multisig Wallet

If you look at the function signature ```submitTransaction(address destination, uint value, bytes data)``` you notice a ```bytes data``` field. It is used to invoke functionalities of a contract and can be left empty or "0x" for regular value transfers. Remember that invoking SC functions is just a regular transaction containing the relevant calldata. Calldata is obtained by encoding the desired function's signature and its parameters, but fortunately web3 libraries already do the heavy lifting for you, so no need to do this manually. The recipient address needs to be the address of the Smart Contract whose method you want to invoke.

In this case we want to invoke the ```changeDailyLimit``` function of our wallet Smart Contract. We need the ABI to encode the calldata easily.

In [ ]:
let txHash3 
let callData

In [ ]:
txHash3 
callData = myWallet.methods.changeDailyLimit(web3.utils.toWei("1", "ether")).encodeABI()
myWallet.methods.submitTransaction(myWalletAddress, 0, callData).send({"from": address1})
    .then(x => {
        txHash3 = x
        transactionId = x.events.Submission.returnValues.transactionId
        console.log("Submission success! Transaction ID:" + transactionId)
    })

After this points everything goes as with any other transaction from our wallet

We confirm it with another account as well, which triggers the transaction

In [ ]:
myWallet.methods.confirmTransaction(transactionId).send({"from": address2, "gas": 800000})

In [ ]:
myWallet.methods.transactions(transactionId).call().then(tx => {
    if(tx.executed == true) {
        console.log("Transaction " + transactionId + " is executed")
        myWallet.methods.dailyLimit().call().then(x => console.log(`Daily withdraw limit w/o confirmations: ${x} wei`))
    } else {
        console.log("Transaction " + transactionId + "is not executed")
    }
})


You should see that the daily limit has changed

### 6. Withdraw some ether w/o confirmation
##### Now that the daily limit is changed, it is time to test it. Calldata is 0.

In [ ]:
let  txHash4

In [ ]:
myWallet.methods.submitTransaction(address1, web3.utils.toWei("0.7", "ether"), "0x").send({"from": address1})
    .then(x => {
        txHash4 = x
        transactionId = x.events.Submission.returnValues.transactionId
        console.log("Submission success! Transaction ID:" + transactionId)
    })

In [ ]:
myWallet.methods.transactionCount().call().then((x) => console.log(`Transaction count: ${x}`))
myWallet.methods.transactions(transactionId).call().then((x) => console.log(`Our transaction status: ${JSON.stringify(x)}`))
myWallet.methods.confirmations(transactionId, address1).call().then((x) => console.log(`Confirmed by ${address1}: ${x}`))
myWallet.methods.confirmations(transactionId, address2).call().then((x) => console.log(`Confirmed by ${address2}: ${x}`))
myWallet.methods.confirmations(transactionId, address3).call().then((x) => console.log(`Confirmed by ${address3}: ${x}`))

##### You can see that the transaction is executed, but only 1 account confirmed it. Let's check the remaining daily quota.
The daily quota is calculated for "today" by comparing the current time to a unix timestamp called ```lastDay```. If the curent moment is past ```lastDay + 24 hours``` then the daily quota resets. The lastDay timestamp is initially zero and is first set in the contract when we try to make a withdrawal.

In [ ]:
myWallet.methods.dailyLimit().call().then(x => console.log(`Daily withdraw limit w/o confirmations: ${x} wei`))
myWallet.methods.calcMaxWithdraw().call().then(x => console.log(`Allowed withdraw for today w/o confirmations: ${x} wei`))
myWallet.methods.lastDay().call().then(x => console.log(`Last day: ${x} -> ${new Date(x*1000)}`))

### 7. Let's remove an owner
Steps are very similar to changing the daily limit. You can manage ownership with```removeOwner(address owner)``` and```replaceOwner(address owner, address newOwner)``` methods, and change the confirmations needed with ```changeRequirement(uint _required)```.

I remove owner n3 in this example

In [ ]:
callData = myWallet.methods.removeOwner(address3).encodeABI()
myWallet.methods.submitTransaction(myWalletAddress, 0, callData).send({"from": address1})
    .then(x => {
        txHash5 = x
        transactionId = x.events.Submission.returnValues.transactionId
        console.log("Submission success! Transaction ID:" + transactionId)
    })

In [ ]:
myWallet.methods.confirmTransaction(transactionId).send({"from": address2, "gas": 800000})

In [ ]:
myWallet.methods.transactions(transactionId).call().then(tx => {
    if(tx.executed == true) {
        console.log("Transaction " + transactionId + " is executed")
        myWallet.methods.dailyLimit().call().then(x => console.log(`Daily withdraw limit w/o confirmations: ${x} wei`))
    } else {
        console.log("Transaction " + transactionId + "is not executed")
    }
})


##### You should see the chosen owner disappeared from the list

### 8. Your experiments here
Feel free to play around with your Multisig wallet